# Summary: Genetic Search Algorithm

This algorithm approximates solutions by modeling natural selection. A population of binary strings of a given length is randomly generated. The fitness of each of these solutions is evaluated. The most fit are selected and used to generate a new population. Once the population is created, the algorithm is reiterated on the new population. This is repeated a number of times and the most fit solution is returned.  


Below is a python implementation.

In [154]:
import numpy


# Generate Population of n_pop members with DNA of n_len
def population(n_pop, n_len):
    return ["".join([str(s) for s in numpy.random.randint(0, 2, n_len)]) for _ in range(n_pop)]


# Evaluate the fitness of each member of a population
def fitness(pop, domain):
    return [gfit(c, domain) for c in pop]

# Calculate fitness of a given genotype 
def gfit(indv, domain):
    z = decode(indv, domain)
    return f(z)



# Selection: Simulate tournaments of k=3 over a population
def selection(pop, fit, interval, k=3):
    return [tournament(pop, fit, k) for _ in range(len(pop))]

# Simulate a tournament between k individuals 
# Returns winner
def tournament(pop, fit, interval, k=3):
    # choose random winner
    v_ix = numpy.random.randint(len(pop))
    # competition
    for ix in numpy.random.randint(0, len(pop)-1, k-1):
        if fit[ix] < fit[v_ix]:
            v_ix = ix
    return pop[v_ix]
    

# Cross-over
def crossover(p1, p2, r_cross):
    # define children
    c1, c2 = p1, p2
    # determine if a crossover happens
    if numpy.random.rand() < r_cross:
        # random crossover point
        pt = numpy.random.randint(0, len(c1))
        # crossover
        c1 = p1[:pt] + p2[pt:]
        c1 = p1[pt:] + p2[:pt]
    return c1, c2


# Mutate
def mutate(indv, r_mut):
    f_indv = ""
    for i in range(len(indv)):
        if numpy.random.rand() < r_mut:
            f_indv += str((int(indv[i]) + 1) % 2)
        else:
            f_indv += indv[i]
    return f_indv 


# Get the best 
def best(pop, fit, domain):
    hi_fit = min(fit)
    hi_ix = fit.index(hi_fit)
    return [decode(pop[hi_ix], domain), hi_fit]


# Measure the distribution of genes (debugging)
def distribution(pop, interval, buckets):
    dist = {}
    delta = float(interval[1] - interval[0]) / buckets
    base = 0.0
    for i in range(buckets):
        top = base + delta
        freq = [i for i in pop if (base < decode(i, interval) and decode(i, interval) <top)]
        dist[str(base) + "-" + str(base + delta)] = len(freq)
        base = top
    return [print(str(key) + " : "+ str(value)) for key, value in dist.items()]


# Simulation
def genetic_simulation(f, domain, n_pop, n_len, r_cross, r_mut, n_gen):
    # initialize population
    pop = population(n_pop, n_len)
    for gen in range(n_gen):
        # assign fitness scores
        fit = fitness(pop, domain)

        b = best(pop, fit, domain)
        
        # select reproducing individuals
        vic = selection(pop, fit, domain)
        # initialize next generation
        nxt_pop = []
        # reproduce
        for ps in range(0, n_pop, 2):
            c1, c2 = crossover(vic[ps], vic[ps+1], r_cross)
            c1, c2 = mutate(c1, r_mut), mutate(c2, r_mut)
            nxt_pop.append(c1)
            nxt_pop.append(c2)
        # update population
        pop = nxt_pop
    return b


## These functions define the problem, they are editable
### * f(x) is the cost function to be minimized
### * decode(dna, domain) maps each dna combination to a value in the problem domain

In [155]:

# Function to be minimized [Problem to be solved]
def f(x):
    return (x**3 - 2)**2

# Decoder: Map DNA (binary) to a domain of reals, given as a list containing the endpoints.
def decode(dna, domain):
    # binary to decimal
    dna_dec = int(dna, 2)
    # scale to the domain
    return dna_dec / 2**len(dna) * (domain[1] - domain[0]) + domain[0]


## Hyperparameters
### These values control the simulation's behavior

In [156]:
# Hyperparameters
n_pop = 100
n_len = 64
r_cross = 0.5
r_mut = 1.0 / n_len
n_gen = 100
domain = [-10, 10]

## Sandbox
### The area below is to experiment with the simulation

In [157]:


SOL = numpy.power(2, 1/3)


for i in range(10):
    sol = genetic_simulation(f, domain, n_pop, n_len, r_cross, r_mut, n_gen)
    print("Final from " + str(i + 1) + ": f(" + str(sol[0]) + ") = " + str(sol[1]))
    error = (SOL - sol[0]) / sol[0]
    print("Error: " + str(error) + "%")
    print("--------------")

Final from 1: f(1.2599205970290424) = 4.651088815206767e-12
Error: 3.5943997733148615e-07%
--------------
Final from 2: f(1.249999999949825) = 0.0021972656470495497
Error: 0.007936839956357118%
--------------
Final from 3: f(1.2599210592041956) = 1.9654046852124062e-15
Error: -7.388814049925365e-09%
--------------
Final from 4: f(1.2597656249927613) = 5.477090159153117e-07
Error: 0.00012337604632825083%
--------------
Final from 5: f(1.2499999999996945) = 0.0021972656251342676
Error: 0.00793683991614492%
--------------
Final from 6: f(1.2599182128828073) = 1.825308376823295e-10
Error: 2.2517430392928247e-06%
--------------
Final from 7: f(1.249999999964512) = 0.0021972656405953375
Error: 0.007936839944514329%
--------------
Final from 8: f(1.259921193129614) = 4.65278160937433e-13
Error: -1.1368547613606116e-07%
--------------
Final from 9: f(1.2597656249876632) = 5.477090518419835e-07
Error: 0.00012337605037564904%
--------------
Final from 10: f(1.2499999999946638) = 0.00219726562734